# Poisson with Nitsche boundary conditions

Solving $-\Delta p = f$, and imposing bc with nitsche.

In [ ]:
import dolfinx
import dolfinx.fem
import dolfinx.mesh
import mpi4py.MPI
import numpy as np
import ufl

from ufl import (FiniteElement, Measure, VectorElement, MixedElement, SpatialCoordinate,
                 TestFunction, TrialFunction, div, grad, exp, inner, FacetNormal,
                 Circumradius)
from dolfinx import fem, io, mesh

In [ ]:
n = 20
m = 25
domain = dolfinx.mesh.create_unit_square(mpi4py.MPI.COMM_WORLD, n, m)

In [ ]:
k = 2
V_el = FiniteElement("CG", domain.ufl_cell(), k)
V = fem.FunctionSpace(domain, V_el)

In [ ]:
uD = fem.Function(V)
x = SpatialCoordinate(domain)

u_ex =  1 + x[0]**2 + 2 * x[1]**2

uD.interpolate(fem.Expression(u_ex, V.element.interpolation_points()))
f = -div(grad(u_ex))

In [ ]:
u = TrialFunction(V)
v = TestFunction(V)
alpha = 1000

dx = ufl.dx
ds = ufl.ds
n = ufl.FacetNormal(domain)
h = ufl.Circumradius(domain)

a  = inner(grad(u), grad(v)) * dx
a += -inner(grad(u), n) * v * ds
a += -inner(grad(v), n) * u * ds
a += alpha/h*inner(u,v) * ds

L  = f * v * dx
L += -inner(grad(v), n) * uD * ds 
L += alpha/h*inner(uD,v) * ds

In [ ]:
problem = fem.petsc.LinearProblem(a, L)

In [ ]:
try:
    u_h = problem.solve()
except PETSc.Error as e:
    if e.ierr == 92:
        print("The required PETSc solver/preconditioner is not available. Exiting.")
        print(e)
        exit(0)
    else:
        raise e

u_h.name = 'u'
uD.name = 'u_exact'

In [ ]:
from dolfinx import io
# with io.VTXWriter(domain.comm, "output.bp", [w_h]) as vtx:
#     vtx.write(0.0)
with io.XDMFFile(domain.comm, "poisson_nitsche.xdmf", "w") as xdmf:
    xdmf.write_mesh(domain)
    xdmf.write_function(u_h)
    xdmf.write_function(uD)

In [ ]:
import numpy as np

error_form_u = fem.form(inner(u_h-u_ex, u_h-u_ex) * dx)
error_local_u = fem.assemble_scalar(error_form_u)
errorL2_u = np.sqrt(domain.comm.allreduce(error_local_u, op=mpi4py.MPI.SUM))

error_form_1 = fem.form(inner(grad(u_h-u_ex), grad(u_h-u_ex)) * dx)
error_local_u_1 = fem.assemble_scalar(error_form_1)
errorH1_u = np.sqrt(domain.comm.allreduce(error_local_u_1, op=mpi4py.MPI.SUM))

if domain.comm.rank == 0:
    print(fr"$L^2$-error u: {errorL2_u:.2e}")
    print(fr"$H^1$-error u: {errorH1_u:.2e}")

k = 1 

$L^2$-error u: 9.80e-04

$H^1$-error u: 5.45e-02


k = 2

$L^2$-error u: 2.26e-04

$H^1$-error u: 1.44e-03